# LLM Aided Retrieval
Usa a LLM para fazer a filtragem de documentos

In [12]:
from langchain_openai import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


embedding = OpenAIEmbeddings()

chroma_directory = "../documents/chroma_vectorstore"

metadata_info = [
    AttributeInfo(
        name="source",
        description="Nome da apostila de onde o texto original foi retirado.",
        type="string"
    ),
    AttributeInfo(
        name="page",
        description="A página da apostila de onde o texto se origina.",
        type="integer"
    )
]

document_description = "Apostilas de cursos"

llm = OpenAI()

paths = [
    "../documents/ApostilaLangChain.pdf",
    "../documents/Explorando a API da OpenAI.pdf",
    "../documents/Explorando o Universo das IAs com Hugging Face.pdf"
]

pages = []

for path in paths:
    loader = PyPDFLoader(path)
    pages.extend(loader.load())
    
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50,
    separators=["\n\n", "\n",  " ", ""]
)

documents = splitter.split_documents(pages)

vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=chroma_directory
)

retriever = SelfQueryRetriever.from_llm(
    llm,
    vector_store,
    document_description,
    metadata_info,
    verbose=True
)

In [17]:
from langchain.globals import set_debug

set_debug(True)

question = 'O que a apostila do HuggingFace fala sobre a OpenAI e o ChatGPT?'

documents_result = retriever.get_relevant_documents(question)

for doc in documents_result:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

[chain/start] [retriever:Retriever > chain:query_constructor] Entering Chain run with input:
{
  "query": "O que a apostila do HuggingFace fala sobre a OpenAI e o ChatGPT?"
}
[chain/start] [retriever:Retriever > chain:query_constructor > prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "O que a apostila do HuggingFace fala sobre a OpenAI e o ChatGPT?"
}
[chain/end] [retriever:Retriever > chain:query_constructor > prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:Retriever > chain:query_constructor > llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Your goal is to structure the user's query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n```json\n{\n    \"query\": string \\ text string to compare to document contents\n    \"filter\": string \\ logical condition statemen

In [16]:
question = 'Do que se trata a página 44 da apostila de HuggingFace?'

documents_result = retriever.get_relevant_documents(question)

for doc in documents_result:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
Autenticando com o token do Hugging Face
Para usar modelos restritos, vamos precisar criar uma conta no Hugging Face e pedir por autorização
de uso do modelo. Em seguida, precisamos criar um token de acesso (que não tem nada a ver com os
tokens dos tokenizers ), para que nosso código entenda qual usuário está tentando usar o modelo
e nos dê acesso a ele.
Criando a conta
==========={'page': 44, 'source': '../documents/Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
Autenticando com o token do Hugging Face
Para usar modelos restritos, vamos precisar criar uma conta no Hugging Face e pedir por autorização
de uso do modelo. Em seguida, precisamos criar um token de acesso (que não tem nada a ver com os
tokens dos tokenizers ), para que nosso código entenda qual usuário está tentando usar o modelo
e nos dê acesso a ele.
Criando a conta
==========={'page': 44, 'source': '../documents/Explorand